In [1]:
import os
import time
import threading    # Import the threading module to run the speech recognition in a separate thread
import pyttsx3  # Import the pyttsx3 library to convert text to speech
import speech_recognition as sr    # Import the speech recognition library
import openai  # Import OpenAI API
from haystack.document_stores import InMemoryDocumentStore  # Import InMemoryDocumentStore
from haystack.nodes import DensePassageRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from document import documents  # Import documents from the external file

#my openAI private secret key
openai.api_key = "you open AI key"

# Initialize text-to-speech engine
tts_engine = pyttsx3.init()
tts_engine.setProperty('rate', 150)
tts_engine.setProperty('volume', 1)

# Initialize speech recognizer
recognizer = sr.Recognizer()

# Initialize Haystack's RAG pipeline
document_store = InMemoryDocumentStore(embedding_dim=768)
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=False
)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")
rag_pipeline = ExtractiveQAPipeline(reader, retriever)

# writing and updating from document.py file
document_store.write_documents(documents)
document_store.update_embeddings(retriever)

print("Components initialized successfully.")


Documents Processed: 10000 docs [00:09, 1052.11 docs/s]       

Components initialized successfully.


In [2]:
def listen_to_user():
    # Listen to the user's voice input and convert it to text.
    with sr.Microphone() as source:
        print("Listening...")
        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)  # Listen for 5 seconds
            user_input = recognizer.recognize_google(audio)
            print(f"You said: {user_input}")
            return user_input
        except sr.UnknownValueError:
            print("Sorry, I didn't catch that. Please try again.")
            return None
        except sr.RequestError:
            print("Network error. Please check your connection.")
            return None

def generate_response_with_rag(query):
    # Generate a response using Haystack's RAG model.
    try:
        print("Searching for relevant information...")
        result = rag_pipeline.run(query=query, params={"Retriever": {"top_k": 1}, "Reader": {"top_k": 1}})  # Run the RAG pipeline
        if result["answers"]:
            return result["answers"][0].answer
        else:
            return "I couldn't find any relevant information in my database."
    except Exception as e:  # Catch any errors that occur during the search
        print(f"Error using RAG: {e}")
        return "I encountered an error while searching for an answer."

def generate_response_with_gpt(prompt):
    # Generate a response using OpenAI's GPT-3.5 model. as it is better in scale for perfoamce
    # openAI gpt models perform better than llm so we decided to use the openAI gpt model
    try:
        print("Generating response using GPT-3.5...")
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0.7
        )
        return response["choices"][0]["message"]["content"].strip()  # Return the generated response
    except Exception as e:
        print(f"Error using GPT-3.5: {e}")
        return "I'm sorry, I couldn't generate a response at this time."

def speak_output(response):
    # Speak the response using text-to-speech.
    # we have used threads to run the speech recognition in a separate thread
    try:
        tts_engine.stop()  # Interrupt any ongoing speech
        tts_engine.say(response)  # Speak the response
        tts_engine.runAndWait()  # Wait for the speech to finish
    except RuntimeError as e:
        print(f"Error with text-to-speech: {e}")

def stop_speech():
    # Stop the text-to-speech engine.
    try:
        tts_engine.stop()  # Stop the speech
    except RuntimeError:
        pass


In [3]:
def main():
    #this main function completes the whole process of the chatbot
    print("Bot is ready. You can speak now!")
    speak_output("Hello! I am your assistant. How can I help you today?")
    
    while True:
        user_input = listen_to_user()
        if user_input is None:
            continue

        # to exit the ongoing conversation user can say "exit" or "quit"
        if "exit" in user_input.lower() or "quit" in user_input.lower():
            print("Goodbye!")
            speak_output("Goodbye! Have a great day!")
            break

        # Generate response using RAG
        #documents store all the rag information for faster retrival
        response = generate_response_with_rag(user_input)

        # If RAG fails to find relevant content, model fallbacks to GPT to fetch answer
        if response == "I couldn't find any relevant information in my database.":
            response = generate_response_with_gpt(user_input)

        # Print and speak the bot's response
        print(f"Bot: {response}")
        speak_output(response)


In [4]:
# Start the chatbot
main()


Bot is ready. You can speak now!
Listening...
You said: what is AI
Searching for relevant information...


Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.21s/ Batches]


Bot: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines
Listening...
You said: what is life
Searching for relevant information...


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.06 Batches/s]


Bot: existence
Listening...
You said: what are you doing
Searching for relevant information...


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.17 Batches/s]


Bot: to help you.
Listening...
You said: exit
Goodbye!
